In [11]:
import numpy as np
from matplotlib import pyplot as plt
import os
from load_data import load_data
import cv2 
#visual = __import__ ('3dvis')

dirname = os.path.dirname(os.path.abspath('Ex1'))
data_path = os.path.join(dirname,'data', 'data.p')
data = load_data(data_path)

# extract data
#point cloud
xyz_velodyne = data['velodyne'][:,0:3] #data from 0 to 3-1

#projection matrices
P = data['P_rect_20']
T = data['T_cam2_velo']
#image
image2 = data['image_2']
#labels
sem_label = data['sem_label']
color_map = data['color_map']

#filter points with negative x
velodyne_fltrd = []
sem_label_fltrd = []
for i in range(xyz_velodyne.shape[0]):
    if xyz_velodyne[i, 0] >= 0:
        velodyne_fltrd.append(xyz_velodyne[i, :])
        sem_label_fltrd.append(sem_label[i])
velodyne_fltrd = np.array(velodyne_fltrd)
sem_label_fltrd = np.array(sem_label_fltrd)

#Projection of point cloud in image 2 coordinates
a = np.ones((velodyne_fltrd.shape[0],1))
velodyne_fltrd = np.hstack((velodyne_fltrd, a))
velodyne_fltrd = np.transpose(velodyne_fltrd)

extrin_calib = np.matmul(T,velodyne_fltrd)
proj_cloud = np.matmul(P,extrin_calib)/extrin_calib[2,:] #normalization by Zc

u,v,k = proj_cloud   #k is an array of ones
#for i in range(len(u)):
    #if u[i] > 376 or u[i] < 0 or v[i] > 1241 or v[i] < 0:
        #u[i] = 0
        #v[i] = 0
    #else:
        #print(u[i])
            
print(u)
u = u.astype(np.int32)
v = v.astype(np.int32)



#Draw color point cloud on image
img = image2.astype(np.uint8)
color=np.zeros(velodyne_fltrd.shape[1])
for i in range(velodyne_fltrd.shape[1]):
    label=sem_label_fltrd[i][0]
    color = tuple(color_map.get(label))
    # Draw a circle of corresponding color 
    cv2.circle(img,(u[i],v[i]), 1, color, -1)

#cv2.circle(img,(1200, 300),10, (255,0,0), -1)
cv2.imshow('image2',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[0]
[546.58071526 544.29695572 539.71977822 ... 921.70611441 919.01920834
 917.68746828]
